In [1]:
import openai
import textwrap as _tw
from credentials import api_key
openai.api_key = api_key

MODEL = "gpt-3.5-turbo"

MODEL4 = 'gpt-4'

Verify working


In [2]:


initial_messages = [{"role":"system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": "Why is the sky blue?"},
                    {"role": "assistant", "content" : "The sky is not really blue, but "}]
completion = openai.ChatCompletion.create(
    model=MODEL,
    messages= initial_messages)
print(_tw.fill(completion.choices[0].message.content, width=100))

it appears blue to us because of a phenomenon called Rayleigh scattering. This is when the Earth's
atmosphere scatters sunlight in all directions, and blue light is scattered more than other colors
because it travels in smaller and shorter wavelengths. As a result, we see blue light more
frequently and the sky appears blue during the daytime.


## Goals

* Generate a story and then interact with it by interjecting changes to it. 

*  TODO:  Allow use to back up a bit somehow. 

*  TODO: Allow user to save/load a story.

* TODO: Add a method to create a story summary for continuation.

In [36]:
class gpt_story:
    def __init__(self, topic = "anything", model = "gpt-4", max_tokens = 100):
        self.model = model
        self.max_tokens = max_tokens
        self.messages = [{"role":"system", "content": "You are a creative writer."},
                                 {"role": "user", "content": f"Write a story about {topic}."}]
        completion = openai.ChatCompletion.create(
            model=self.model,
            messages= self.messages, max_tokens = self.max_tokens)
        self.messages = self.messages + [dict(completion.choices[0].message)]
        
    def continue_with(self, message):
        message_dict = {'role':'assistant', 'content' : message}
        self.messages = self.messages +  [message_dict]
        completion = openai.ChatCompletion.create(
            model=self.model,
            messages= self.messages, max_tokens = self.max_tokens)
        self.messages = self.messages + [dict(completion.choices[0].message)]
        
    
    def __repr__(self):
        return _tw.fill(' '.join([message['content'] for message in self.messages]), width=120)
    
    def __len__(self):
        return len(self.messages)
    
    def __getitem__(self, key):
        return self.messages[key]
    
    def __iter__(self):
        return iter(self.messages)
 

In [37]:
test = gpt_story("sailing", max_tokens=50)
test 

You are a creative writer. Write a story about sailing. Once upon a time, in a calm coastal town, nestled between the
sun and the sea, there was a sailor named Thomas. Thomas was a vibrant and hearty man, brimming with life, and his
greatest passion was sailing. Now, this wasn

In [38]:
test.continue_with("'t the greatest sailor, but he made up for it with passion.")
test

You are a creative writer. Write a story about sailing. Once upon a time, in a calm coastal town, nestled between the
sun and the sea, there was a sailor named Thomas. Thomas was a vibrant and hearty man, brimming with life, and his
greatest passion was sailing. Now, this wasn 't the greatest sailor, but he made up for it with passion. The townspeople
loved listening to the tales of his adventures, as he navigated his small wooden boat, named Serenity, to faraway lands
and discovered new treasures. Much like the changing tides that lapped upon the shores of their coastal haven

In [39]:
len(test)

5